<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-Pytorch/blob/main/CLIP_Zero_Shot_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use CLIP Zero-Shot on your own classificaiton dataset

This notebook provides an example of how to benchmark CLIP's zero shot classification performance on your own classification dataset.

[CLIP](https://openai.com/blog/clip/) is a new zero shot image classifier relased by OpenAI that has been trained on 400 million text/image pairs across the web. CLIP uses these learnings to make predicts based on a flexible span of possible classification categories.

CLIP is zero shot, that means **no training is required**. 

Try it out on your own task here!

Be sure to experiment with various text prompts to unlock the richness of CLIP's pretraining procedure.


# Download and Install CLIP Dependencies

In [ ]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

import numpy as np
import torch
import os

print("Torch version:", torch.__version__)
os.kill(os.getpid(), 9)
#Your notebook process will restart after these installs

CUDA version: 11.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.4MB/s eta 0:03:54tcmalloc: large alloc 1147494400 bytes == 0x55cc60c8e000 @  0x7f020df1a615 0x55cc271d2cdc 0x55cc272b252a 0x55cc271d5afd 0x55cc272c6fed 0x55cc27249988 0x55cc272444ae 0x55cc271d73ea 0x55cc272497f0 0x55cc272444ae 0x55cc271d73ea 0x55cc2724632a 0x55cc272c7e36 0x55cc27245853 0x55cc272c7e36 0x55cc27245853 0x55cc272c7e36 0x55cc27245853 0x55cc272c7e36 0x55cc2734a3e1 0x55cc272aa6a9 0x55cc27215cc4 0x55cc271d6559 0x55cc2724a4f8 0x55cc271d730a 0x55cc272453b5 0x55cc272447ad 0x55cc271d73ea 0x55cc272453b5 0x55cc271d730a 0x55cc272453b5
     |█████████████████████████████▏  | 1055.7MB 1.5MB/s eta 0:01:07tcmalloc: large alloc 1434370048 bytes == 0x55cca52e4000 @  0x7f020df1a615 0x55cc271d2cdc 0x55cc272b252a 0x55cc271d5afd 0x55cc272c6fed 0x55cc27249988 0x55cc272444ae 0x55cc271d73ea 0x55cc272497f0 0x55cc272444ae 0x55cc271d73ea 0x55cc2724632a 0x55cc272c7e

In [1]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 90, done.
remote: Total 90 (delta 0), reused 0 (delta 0), pack-reused 90
Unpacking objects: 100% (90/90), done.
/content/CLIP


# Download Classification Data or Object Detection Data

We will download the [public flowers classificaiton dataset](https://public.roboflow.com/classification/flowers_classification) from Roboflow. The data will come out as folders broken into train/valid/test splits and seperate folders for each class label.

You can easily download your own dataset from Roboflow in this format, too.

We made a conversion from object detection to CLIP text prompts in Roboflow, too, if you want to try that out.


To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [ ]:
#download classification data
#replace with your link
!curl -L "https://public.roboflow.com/ds/iwFPJ4BJdO?key=M0lehxxyds" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [3]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./test/')
class_names.remove('_tokenization.txt')
class_names

['dandelion', 'daisy']

In [4]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./test/_tokenization.txt

An example picture from the Flowers_Classification dataset depicting a daisy
An example picture from the Flowers_Classification dataset depicting a dandelion

In [5]:
#edit your prompts as you see fit here, be sure the classes are in teh same order as above
%%writefile ./test/_tokenization.txt
An example picture from the flowers dataset depicting a daisy
An example picture from the flowers dataset depicting a dandelion

Overwriting ./test/_tokenization.txt


In [6]:
candidate_captions = []
with open('./test/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

# Run CLIP inference on your classification dataset

In [7]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./test/' + cls + '/*.jpg')
    for img in test_imgs:
        #print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            #print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
print('accuracy on all is : ' + str(sum(correct)/len(correct)))

100%|███████████████████████████████████████| 354M/354M [00:06<00:00, 55.7MiB/s]


accuracy on class dandelion is :0.009523809523809525
accuracy on class daisy is :0.05194805194805195
accuracy on all is : 0.027472527472527472


In [ ]:
#Hope you enjoyed!
#As always, happy inferencing
#Roboflow